<div style="text-align: center;"> <h3>Data Mining and Wrangling</h3>
<h5>Formative Assessment 3</h5>
<h5>February 23, 2025</h5>    
<h5><u>By Romand Lansangan</u></h5>
    </div>
    
---

### Case study: Major League Baseball
What is the relationship between payroll and wins among Major League Baseball (MLB) teams? In this homework,
we’ll find out by wrangling, exploring, and modeling the dataset in MLPayData_Total.rdata, which contains the
winning records and the payroll data of all 30 MLB teams from 1998 to 2014.
The dataset has the following variables:
* payroll: total team payroll (in billions of dollars) over the 17-year period
* avgwin: the aggregated win percentage over the 17-year period
* Team.name.2014: the name of the team
* p1998, . . . , p2014: payroll for each year (in millions of dollars)
* X1998, . . . , X2014: number of wins for each year
* X1998.pct, . . . , X2014.pct: win percentage for each year
We’ll need to use the following R packages:

<style>
    .jp-OutputArea-child {
        display: flex;
        justify-content: center;
    }
</style>

In [216]:
library(tidyverse)
library(tibble)
library(ggrepel)
library(ggthemes)
library(knitr)
library(formattable)
library(IRdisplay)
library(grid)

### Wrangle
#### 1.1 Import (5 points)
* Import the data into a tibble called mlb_raw and print it.

<center>

In [2]:
load("ml_pay.rdata")

In [4]:
mlb_raw <- tibble(ml_pay)
print_table <- function(df, head=7){
    formatted_table <- df %>%
        head(head) %>%
        formattable()

    display_html(paste0("<div style='display: flex; justify-content: center;'>", as.character(formatted_table), "</div>"))
}
print_table(mlb_raw[,c(1:10)])

payroll,avgwin,Team.name.2014,p1998,p1999,p2000,p2001,p2002,p2003,p2004
1.120874,0.4902585,Arizona Diamondbacks,31.61450,70.49600,81.02783,81.20651,102.82000,80.64033,70.20498
1.381712,0.5527605,Atlanta Braves,61.70800,74.89000,84.53784,91.85169,93.47037,106.24367,88.50779
1.161212,0.4538250,Baltimore Orioles,71.86092,72.19836,81.44743,72.42633,60.49349,73.87750,51.21265
1.972359,0.5487172,Boston Red Sox,59.49700,71.72500,77.94033,109.55891,108.36606,99.94650,125.20854
1.459767,0.4736557,Chicago Cubs,49.81600,42.14276,60.53933,64.01583,75.69083,79.86833,91.10167
1.315391,0.5111170,Chicago White Sox,35.18000,24.53500,31.13350,62.36300,57.05283,51.01000,65.21250
1.024782,0.4861602,Cincinnati Reds,20.70733,73.27846,46.86720,45.22788,45.05039,59.35567,43.06786


<center style="font-size: 17px"><b>Table 1</b>: Table representing original ML Pay dataset</center>

How many rows and columns does the data have?

In [6]:
print(paste("Rows: ", nrow(mlb_raw)))
print(paste("Columns: ", ncol(mlb_raw)))

[1] "Rows:  30"
[1] "Columns:  54"


Does this match up with the data description given above?

In [8]:
temp <- mlb_raw %>%
select(payroll, avgwin, Team.name.2014, p1998, p2014, X1998, X2014, X1998.pct, X2014.pct)
print_table(temp)

payroll,avgwin,Team.name.2014,p1998,p2014,X1998,X2014,X1998.pct,X2014.pct
1.120874,0.4902585,Arizona Diamondbacks,31.61450,112.68867,65,59,0.3987730,0.4154930
1.381712,0.5527605,Atlanta Braves,61.70800,110.89734,106,73,0.6503067,0.5140845
1.161212,0.4538250,Baltimore Orioles,71.86092,107.40662,79,82,0.4846626,0.5774648
1.972359,0.5487172,Boston Red Sox,59.49700,162.81741,92,62,0.5644172,0.4366197
1.459767,0.4736557,Chicago Cubs,49.81600,89.00786,90,64,0.5521472,0.4507042
1.315391,0.5111170,Chicago White Sox,35.18000,91.15925,80,63,0.4907975,0.4436620
1.024782,0.4861602,Cincinnati Reds,20.70733,112.39077,77,66,0.4723926,0.4647887


<center style="font-size: 17px"><b>Table 1a</b>: Table representing original ML Pay dataset with selected columns</center>

Yes, all the columns mentioned above are present, and are consistent with their supposed data type.

#### 1.2 Tidy (15 points)
The raw data are in a messy format: Some of the column names are hard to interpret, we have data from different years in the same row, and both year-by-year and aggregate data are present.
* Tidy the data into two separate tibbles: one called mlb_aggregate containing the aggregate data and another called mlb_yearly containing the year-by-year data. mlb_total should contain columns named team, payroll_aggregate, pct_wins_aggregate and mlb_yearly should contain columns named team, year, payroll, pct_wins, num_wins. Comment your code to explain each step.
* Print these two tibbles. How many rows do mlb_aggregate and mlb_yearly contain, and why?

**Solution**:

In [10]:
mlb_tidy <- mlb_raw %>%
    pivot_longer(colnames(mlb_raw[,4:ncol(mlb_raw)])) %>%
    mutate(year = as.numeric(gsub("\\D", "", name)), var = gsub("[[:digit:]]+", "", name), team=Team.name.2014) %>%
    select(-c("name", "Team.name.2014")) %>%
    pivot_wider(names_from = var, values_from = value) %>%
    rename(payroll_avg = payroll, payroll = p, pct_wins = X.pct, num_wins=X)
print_table(mlb_tidy)

payroll_avg,avgwin,year,team,payroll,num_wins,pct_wins
1.120874,0.4902585,1998,Arizona Diamondbacks,31.61450,65,0.3987730
1.120874,0.4902585,1999,Arizona Diamondbacks,70.49600,100,0.6134969
1.120874,0.4902585,2000,Arizona Diamondbacks,81.02783,85,0.5246914
1.120874,0.4902585,2001,Arizona Diamondbacks,81.20651,92,0.5679012
1.120874,0.4902585,2002,Arizona Diamondbacks,102.82000,98,0.6049383
1.120874,0.4902585,2003,Arizona Diamondbacks,80.64033,84,0.5185185
1.120874,0.4902585,2004,Arizona Diamondbacks,70.20498,51,0.3148148


<center style="font-size: 17px"><b>Table 2</b>: Tidy Version of <i>Table 1</i></center>

The data was cleaned first by applying *pivot longer* function for all the 4th columns and beyond. Then the years are separated in their own variable, named `year` column. For example, a row that has "p2008" was separated so that "p" and "2008" are in its own column. Then the result was pivotted wider based on the `var` where "p", "x", "X.pct" was located. Finally, the variables was renamed for the sake of interpretability.

In [12]:
mlb_total <- mlb_tidy %>%
    group_by(team) %>%
    summarise(payroll_aggregate = mean(payroll), pct_wins_aggregate=mean(pct_wins)) %>%
    select(c("team", "payroll_aggregate", "pct_wins_aggregate"))
print_table(mlb_total)

team,payroll_aggregate,pct_wins_aggregate
Arizona Diamondbacks,71.94024,0.4921264
Atlanta Braves,89.31234,0.5631539
Baltimore Orioles,76.78065,0.4570920
Boston Red Sox,123.74004,0.5512860
Chicago Cubs,91.27798,0.4745898
Chicago White Sox,80.88858,0.5069221
Cincinnati Reds,65.80984,0.4913973


<center style="font-size: 17px"><b>Table 2a</b>: Aggregated version of ML Pay by mean <i>payroll</i> and <i>pct_wins</i></center>

In [14]:
print(paste("Rows: ", nrow(mlb_total)))
print(paste("Columns: ", ncol(mlb_total)))

[1] "Rows:  30"
[1] "Columns:  3"


In [16]:
mlb_yearly <- mlb_tidy %>%
    select(c("team", "year", "payroll","pct_wins", "num_wins"))
print_table(mlb_yearly)

team,year,payroll,pct_wins,num_wins
Arizona Diamondbacks,1998,31.61450,0.3987730,65
Arizona Diamondbacks,1999,70.49600,0.6134969,100
Arizona Diamondbacks,2000,81.02783,0.5246914,85
Arizona Diamondbacks,2001,81.20651,0.5679012,92
Arizona Diamondbacks,2002,102.82000,0.6049383,98
Arizona Diamondbacks,2003,80.64033,0.5185185,84
Arizona Diamondbacks,2004,70.20498,0.3148148,51


<center style="font-size: 17px"><b>Table 2c</b>: Yearly payroll and win percentage of each team</center>

In [18]:
print(paste("Rows: ", nrow(mlb_yearly)))
print(paste("Columns: ", ncol(mlb_yearly)))

[1] "Rows:  510"
[1] "Columns:  5"


### 1.3 Quality control (15 points)

It’s always a good idea to check whether a dataset is internally consistent. In this case, we are given both aggregated
and yearly data, so we can check whether these match. To this end, carry out the following steps:
* Create a new tibble called mlb_aggregate_computed based on aggregating the data in mlb_yearly, containing columns named team, payroll_aggregate_computed, and pct_wins_aggregate_computed.
* Ideally, mlb_aggregate_computed would match mlb_aggregate. To check whether this is the case, join these two tibbles into mlb_aggregate_joined (which should have five columns: team, payroll_aggregate, pct_wins_aggregate, payroll_aggregate_computed, and pct_wins_aggregate_computed.)
* Create scatter plots of payroll_aggregate_computed versus payroll_aggregate and pct_wins_aggregate_computed versus pct_wins_aggregate, including a 45◦ line in each. Display these scatter plots side by side, and comment on the relationship between the computed and provided aggregate statistics.

In [20]:
mlb_aggregate_computed <- mlb_yearly %>%
    group_by(team) %>%
    summarise(payroll_aggregate_computed = mean(payroll), pct_wins_aggregate_computed=mean(pct_wins))
print_table(mlb_aggregate_computed)

team,payroll_aggregate_computed,pct_wins_aggregate_computed
Arizona Diamondbacks,71.94024,0.4921264
Atlanta Braves,89.31234,0.5631539
Baltimore Orioles,76.78065,0.4570920
Boston Red Sox,123.74004,0.5512860
Chicago Cubs,91.27798,0.4745898
Chicago White Sox,80.88858,0.5069221
Cincinnati Reds,65.80984,0.4913973


<center style="font-size: 17px"><b>Table 2c.1</b>: Aggregated version of <i>Table 2c</i> by mean <i>payroll</i> and <i>pct_wins</i></center>

In [22]:
combined <- merge(mlb_total, mlb_aggregate_computed, by.x = "team", by.y = "team")

In [76]:
print_plot <- function(filename, size=900){
    html_code <- paste0(
      "<div style='display: flex; justify-content: center;'>
          <img src='", filename, "' width='",size,"px' style='margin: auto;'>
       </div>"
    )
    display_html(html_code)
}

In [220]:
options(repr.plot.width = 15, repr.plot.height = 8)

p <- ggplot(combined, aes(x=payroll_aggregate_computed, y=payroll_aggregate)) +
geom_point(size = 3, color = "#0072B2", alpha = 0.7) + 
geom_smooth(method = "lm", formula = y ~ x,color = "red", linetype = "dashed", se = FALSE, aes(color = "Regression Line (45°)")) +
ggtitle("Payroll Aggregate Computed vs Payroll Aggregate") +
xlab("Payroll Computed [millions]") +
ylab("Payroll Actual [millions]") +
scale_color_manual(name = "Legend", values = c("Regression Line (45°)" = "red")) +
theme_grey() +
theme(
    plot.title = element_text(size = 22, face = "bold", hjust = 0.5), 
    axis.title = element_text(size = 18, face = "bold"),
    axis.title.x = element_text(margin = margin(t = 10)),
    axis.title.y = element_text(margin = margin(r = 15)),
    axis.text = element_text(size = 14),
    legend.title = element_text(size = 16, face = "bold"),
    legend.text = element_text(size = 14),
    legend.position = "bottom",
    panel.grid.major = element_line(color = "grey80"), 
    panel.grid.minor = element_blank()
  )
filename = "fig1.png"
ggsave(filename, p, width=15, height = 8)
print_plot(filename)

<center style="font-size: 17px"><b>Figure 1</b>: Scatter plot of `payroll` from <i>Table 2a</i> and <i>Table 2c.1</i> with Simple Regression Line (Red)  Fitted</center>

In [222]:
p <- ggplot(combined, aes(x = pct_wins_aggregate_computed, y = pct_wins_aggregate)) +
    geom_point(size = 3, color = "#0072B2", alpha = 0.7) + 
    geom_smooth(method = "lm", formula = y ~ x, linetype = "dashed", se = FALSE, aes(color = "Regression Line (45°)")) +
    ggtitle("PCT Wins Computed vs PCT Wins Aggregate") +
    xlab("PCT Wins Computed") +
    ylab("PCT Wins Actual") +
    scale_color_manual(name = "Legend", values = c("Regression Line (45°)" = "red")) +
    theme_grey() +
    theme(
        plot.title = element_text(size = 22, face = "bold", hjust = 0.5), 
        axis.title = element_text(size = 18, face = "bold"),
        axis.title.x = element_text(margin = margin(t = 10)),
        axis.title.y = element_text(margin = margin(r = 15)),
        axis.text = element_text(size = 14),
        legend.title = element_text(size = 16, face = "bold"),
        legend.text = element_text(size = 14),
        legend.position = "bottom",
        panel.grid.major = element_line(color = "grey80"), 
        panel.grid.minor = element_blank()
    )
filename = "fig2.png"
ggsave(filename, p, width=15, height = 8)
print_plot(filename)

<center style="font-size: 17px"><b>Figure 2</b>: Scatter plot of `pct_wins` from <i>Table 2a</i> and <i>Table 2c.1 with Simple Regression Line (Red)  Fitted</i></center>

Since all points in **Figure 1** and **Figure 2** aligns in the *45° line*, it becomes clear the values aggregated of `payroll` and `pct_wins` from original data (**Table 2a**) and yearly data (**Table 2c.1**) are consistent/identical. This is a great indicator that there occur no error in the computation of both dataset.

### 2 Explore (50 points for correctness; 10 points for presentation)
Now that the data are in tidy format, we can explore them by producing visualizations and summary statistics.
#### 2.1 Payroll across years (15 points)
* Plot payroll as a function of year for each of the 30 teams, faceting the plot by team and adding a red dashed horizontal line for the mean payroll across years of each team.
* Using dplyr, identify the three teams with the greatest payroll_aggregate_computed, and print a table of these teams and their payroll_aggregate_computed.
* Using dplyr, identify the three teams with the greatest percentage increase in payroll from 1998 to 2014 (call it pct_increase), and print a table of these teams along with pct_increase as well as their payroll figures from 1998 and 2014.
* How are the metrics payroll_aggregate_computed and pct_increase reflected in the plot above, and how can we see that the two sets of teams identified above are the top three in terms of these metrics?


In [80]:
options(repr.plot.width = 20, repr.plot.height = 15)


team_means <- mlb_yearly %>%
  group_by(team) %>%
  summarise(mean_payroll = mean(payroll, na.rm = TRUE))

p <- ggplot(mlb_yearly, aes(x = year, y = payroll, color = team)) +
    geom_point() + 
    geom_hline(data = team_means, aes(yintercept = mean_payroll), color = "red", linetype='dashed') +
    facet_wrap(~team) +
    ggtitle("Payroll [millions] Across Years (Faceted by Team)") +
    theme_grey() + 
    theme(legend.position="none", 
         plot.title = element_text(size = 22, face = "bold", hjust = 0.5),
         strip.text = element_text(size = 16, face = "bold"))
filename = "fig3.png"
ggsave(filename, p, width=15, height = 8)
print_plot(filename, size = 1000)

<center style="font-size: 17px"><b>Figure 3</b>: Scatter plot of `payroll` from <i>Table 2c</i> with with horizontal line (red dashed line) indicating the average `payroll` for each respective team</i></center>

In [62]:
tib <- mlb_aggregate_computed %>%
    arrange(desc(payroll_aggregate_computed)) %>%
    select(team,payroll_aggregate_computed) %>%
    head(3)
print_table(tib)

team,payroll_aggregate_computed
New York Yankees,168.0643
Boston Red Sox,123.7400
Los Angeles Dodgers,110.2467


<center style="font-size: 17px"><b>Table 3a</b>: Top 3 Teams with highest `payroll`</center>

In [64]:
tib <- mlb_yearly %>%
    filter(year %in% c(1998, 2014)) %>%
    select(team, year, payroll) %>%
    pivot_wider(names_from = year, values_from = payroll, names_prefix = "payroll_") %>%
    mutate(pct_increase = ((payroll_2014 - payroll_1998) / payroll_1998)*100) %>%
    arrange(desc(pct_increase)) %>%
    head(3)
print_table(tib)

team,payroll_1998,payroll_2014,pct_increase
Washington Nationals,8.3170,134.7044,1519.6277
Detroit Tigers,19.2375,162.2285,743.2932
Philadelphia Phillies,28.6225,180.0527,529.0601


<center style="font-size: 17px"><b>Table 3b</b>: Top 3 Teams with highest `pct_increase` in `payroll` from 1998-2014</center>

Note that `pct_increase` is calculated as
$$
payroll\_pct\_increase = \frac{payroll_{2014} - payroll_{1998}}{ payroll_{1998}} \times 100
$$

##### How are the metrics payroll_aggregate_computed and pct_increase reflected in the plot above, and how can we see that the two sets of teams identified above are the top three in terms of these metrics?

**Table 3a** shows the top 3 teams with highest `payroll` through the 17 year stretch. These 3 teams—*New York Yankees*, *Boston Red Sox* and *Los Angeles Dodgers*—could be glaringly distinguished from the rest of the teams in **Figure 3** by looking at the teams that have the highest *average horizontal line* (red dashed line).

On the other hand, **Table 3b** shows the top 3 teams with highest `payroll percentage increase` through the 17 year stretch. These 3 teams—*Washington Nationals*, *Detroit Tigers* and *Philadelphia Phillies*—are the three teams experienced the most significant relative payroll growth. They could be distinguished in **Figure 3** by looking at the team that have the highest difference (dispersion) between left most data point (1998) and right most data point (2014).

#### 2.2 Win percentage across years (15 points)
* Plot pct_wins as a function of year for each of the 30 teams, faceting the plot by team and adding a red dashed horizontal line for the average pct_wins across years of each team.
* Using dplyr, identify the three teams with the greatest pct_wins_aggregate_computed and print a table of these teams along with pct_wins_aggregate_computed.
* Using dplyr, identify the three teams with the most erratic pct_wins across years (as measured by the standard deviation, call it pct_wins_sd) and print a table of these teams along with pct_wins_sd.
* How are the metrics pct_wins_aggregate_computed and pct_wins_sd reflected in the plot above, and how can we see that the two sets of teams identified above are the top three in terms of these metrics?

In [100]:
options(repr.plot.width = 20, repr.plot.height = 15)


team_means <- mlb_yearly %>%
  group_by(team) %>%
  summarise(mean_pct_win = mean(pct_wins, na.rm = TRUE))

p <- ggplot(mlb_yearly, aes(x = year, y = pct_wins, color = team)) +
    geom_point() + 
    geom_hline(data = team_means, aes(yintercept = mean_pct_win), color = "red", linetype='dashed') +
    facet_wrap(~team) +
    ggtitle("Win Percentage Across Years (Faceted by Team)") +
    theme_grey() + 
    theme(legend.position="none", 
         plot.title = element_text(size = 22, face = "bold", hjust = 0.5),
         strip.text = element_text(size = 16, face = "bold"))
filename = "fig4.png"
ggsave(filename, p, width=15, height = 8)
print_plot(filename, size = 1000)

<center style="font-size: 17px"><b>Figure 4</b>: Scatter plot of `pct_win` from <i>Table 2c</i> with horizontal line (red dashed line) indicating the average `pct_win` for each respective team</i></center>

In [102]:
tib <- mlb_aggregate_computed %>%
    arrange(desc(pct_wins_aggregate_computed)) %>%
    select(team,pct_wins_aggregate_computed) %>%
    head(3)
print_table(tib)

team,pct_wins_aggregate_computed
New York Yankees,0.5909819
Atlanta Braves,0.5631539
Boston Red Sox,0.5512860


<center style="font-size: 17px"><b>Table 4a</b>: Top 3 Teams with highest `pct_wins`</center>

In [118]:
tib <- mlb_yearly %>%
    filter(year %in% c(1998, 2014)) %>%
    select(team, year, pct_wins) %>%
    pivot_wider(names_from = year, values_from = pct_wins, names_prefix = "pct_wins_") %>%
    mutate(pct_increase = ((pct_wins_2014 - pct_wins_1998) / pct_wins_1998)*100) %>%
    arrange(desc(pct_increase)) %>%
    head(3)
print_table(tib)

team,pct_wins_1998,pct_wins_2014,pct_increase
Miami Marlins,0.3312883,0.4788732,44.54877
Washington Nationals,0.3987730,0.5563380,39.51246
Detroit Tigers,0.3987730,0.5422535,35.98050


<center style="font-size: 17px"><b>Table 4b</b>: Top 3 Teams with highest `pct_increase` in `pct_wins` from 1998-2014</center>

##### How are the metrics pct_wins_aggregate_computed and pct_wins_sd reflected in the plot above, and how can we see that the two sets of teams identified above are the top three in terms of these metrics?

**Table 4a** shows the top 3 MLB teams with highest `pct_wins` or the "winningest" through the 17 year stretch. These 3 teams—*New York Yankees*, *Boston Red Sox* and *Los Angeles Dodgers*—are teams that have manage to stay on top of the league troughout the 17 year period. They could be glaringly distinguished from the rest of the teams in **Figure 3** by looking at the teams that have the highest *average horizontal line* (red dashed line). 

On the other hand, **Table 4b** shows the top 3 teams with highest `winning percentage increase` through the 17 year stretch. These 3 teams—*Miami Marlins*, *Washington Nationals* and *Detroit Tigers*—are the three teams experienced the most significant relative winning growth. They are the team that have manage to achieve the most improvement in terms of winning relative to their winning status during the initial time period. They could be distinguished in **Figure 3** by looking at the team that have the highest difference (dispersion) between left most data point (1998) and right most data point (2014).

#### 2.3 Win percentage versus payroll (15 points)
Let us investigate the relationship between win percentage and payroll.
* Create a scatter plot of pct_wins versus payroll based on the aggregated data, labeling each point with the team name using geom_text_repel from the ggrepel package and adding the least squares line.
* Is the relationship between payroll and pct_wins positive or negative? Is this what you would expect, and why?

In [158]:
options(repr.plot.width = 15, repr.plot.height = 8)

y_mean <- median(mlb_aggregate_computed$pct_wins_aggregate_computed, na.rm = TRUE)
x_mean <- median(mlb_aggregate_computed$payroll_aggregate_computed, na.rm = TRUE)

p <- mlb_aggregate_computed %>%
    ggplot(aes(x = payroll_aggregate_computed, y = pct_wins_aggregate_computed, color = team, label = team)) +
    geom_vline(aes(xintercept = x_mean, linetype = "Payroll Median"), color = "black", linewidth = 1, alpha=0.3) +
    geom_hline(aes(yintercept = y_mean, linetype = "Win % Median"), color = "black", linewidth = 1, alpha=0.3) +
    scale_linetype_manual(name = "Quadrant Lines", values = c("Win % Median" = "dashed", "Payroll Median" = "dashed")) +
    geom_text_repel(show.legend = FALSE) +
    geom_point(size = 3, show.legend = FALSE) + 
    ggtitle("Win Percentage vs Payroll") + 
    ylab("Average Win Percentage") + 
    xlab("Average Payroll [millions]") + 
    theme_gray() + 
    theme(
        legend.position = "top",
        plot.title = element_text(size = 25, hjust = 0.5, face="bold"),
        axis.title = element_text(size=20),
        axis.title.x = element_text(margin = margin(t = 10)),
        axis.title.y = element_text(margin = margin(r = 15))
    )
filename = "fig5.png"
ggsave(filename, p, width=15, height = 8)
print_plot(filename)

<center style="font-size: 17px"><b>Figure 5</b>: Scatter plot of average `pct_win` vs average `payroll`  from <i>Table 2c</i> divided by 4 quadrants based on median</center>

**Figure 5** represents where each MLB team lies in terms of `average payroll` as a function of `average winning percentage`. **Figure 5** could divided into four quadrants. 

The first quadrant (top right), are the teams that have experienced the most winnings but have also spent the most in `payroll`. Also, it is clear that the teams in the first quadrant are the bigger market teams so they could afford to pay for higher payroll and have also experience relative success upon doing do so.

The fourth quadrant (bottom right) are not as succesful compared to the first quadrant given that they are some of the teams that have spent more the the rest of the league. This teams under these quadrant can be seen as inefficient for they spent more than the rest of the league but does not achieve the same return of investment as the other teams.

The second (top left) and third (bottom left) on the other hand are the more efficient team. For the second quadrant, they are teams that have achieve to be above 50th percentile in terms of winnning despite being below 50th percentile in `payroll`. These are some of the teams the could be seen as a potentitial investment opportunities because they are winning more in spite of spending less. It could also be infer that it's only a matter of time before these teams manage to join other bigger markets. Maybe these teams manage to consistently succeed but wasn't been able to become one of the bigger markets due to miriads of reasons like culture in their state, or the team's marketing department. Of course to confirm this suspision, further investigation is imperative. 

Lasly, the third quadrants are the teams that have spent less and won less than the 50th percentile of the legue. These are the teams that could be seen as small market teams and shows a lack of success through the 17 year period. Maybe the reason for them being small market teams are the lack winning results.

**In terms of relationship**, it is clear there is a positive relationship between `payroll` and `win percentage`. Meaning on average, the team that have spent more money on `payroll` also won more. We could confirm this by calculating for correlation and fitting a simple linear regression line.

In [185]:
corr <- cor(mlb_aggregate_computed$payroll_aggregate_computed, mlb_aggregate_computed$pct_wins_aggregate_computed)
print(paste0("Correlation of `payroll` and `pct_win` : ", corr))

[1] "Correlation of `payroll` and `pct_win` : 0.738009593176648"


In [224]:
options(warn = -1)
options(repr.plot.width = 15, repr.plot.height = 8)

y_mean <- median(mlb_aggregate_computed$pct_wins_aggregate_computed, na.rm = TRUE)
x_mean <- median(mlb_aggregate_computed$payroll_aggregate_computed, na.rm = TRUE)

p <- mlb_aggregate_computed %>%
    ggplot(aes(x = payroll_aggregate_computed, y = pct_wins_aggregate_computed, color = team, label = team)) +
    geom_vline(aes(xintercept = x_mean, linetype = "Payroll Median"), color = "black", linewidth = 1, alpha=0.3) +
    geom_hline(aes(yintercept = y_mean, linetype = "Win % Median"), color = "black", linewidth = 1, alpha=0.3) +
    scale_linetype_manual(name = "Quadrant Lines", values = c("Win % Median" = "dashed", "Payroll Median" = "dashed")) +
    geom_text_repel(show.legend = FALSE) +
    geom_point(size = 3, show.legend = FALSE) + 
    geom_smooth(method = "lm",  formula = y ~ x,linetype = "dashed", se = FALSE, color = "red") +
    ggtitle("Win Percentage vs Payroll") + 
    ylab("Average Win Percentage") + 
    xlab("Average Payroll [millions]") + 
    theme_gray() + 
    theme(
        legend.position = "top",
        plot.title = element_text(size = 25, hjust = 0.5, face="bold"),
        axis.title = element_text(size=20),
        axis.title.x = element_text(margin = margin(t = 10)),
        axis.title.y = element_text(margin = margin(r = 15))
    )
filename = "fig5a.png"
ggsave(filename, p, width=15, height = 8)
print_plot(filename)

#### 2.4 Team efficiency (5 points)
Define a team’s efficiency as the ratio of the aggregate win percentage to the aggregate payroll—more efficient teams are those that win more with less money.

* Using dplyr, identify the three teams with the greatest efficiency, and print a table of these teams along with their efficiency, as well as their pct_wins_aggregate_computed and payroll_aggregate_computed.

In [167]:
top_eff <- mlb_aggregate_computed %>%
    mutate(eff = pct_wins_aggregate_computed/payroll_aggregate_computed) %>%
    arrange(desc(eff))
print_table(head(top_eff,3))

team,payroll_aggregate_computed,pct_wins_aggregate_computed,eff
Miami Marlins,41.06429,0.4673161,0.01138011
Oakland Athletics,52.21017,0.5385489,0.01031502
Tampa Bay Rays,45.65805,0.4610341,0.01009754


<center style="font-size: 17px"><b>Table 6</b>: Top 3 most efficient teams based on `pct_wins` over `payroll` from 1998-2014</center>

Note that `Efficieny` is calculated as 
$$
eff = \frac{pct\_wins}{payroll}
$$

In [177]:
mlb_comp_temp <- top_eff %>%
  mutate(team = as.character(team))

mlb_comp_temp[-c(1:3),]$team = ""

y_mean <- median(mlb_aggregate_computed$pct_wins_aggregate_computed, na.rm = TRUE)
x_mean <- median(mlb_aggregate_computed$payroll_aggregate_computed, na.rm = TRUE)

p <- mlb_comp_temp %>%
    ggplot(aes(x = payroll_aggregate_computed, y = pct_wins_aggregate_computed, color = team, label = team)) +
    geom_vline(aes(xintercept = x_mean, linetype = "Payroll Median"), color = "black", linewidth = 1, alpha=0.3) +
    geom_hline(aes(yintercept = y_mean, linetype = "Win % Median"), color = "black", linewidth = 1, alpha=0.3) +
    scale_linetype_manual(name = "Quadrant Lines", values = c("Win % Median" = "dashed", "Payroll Median" = "dashed")) +
    geom_text_repel(show.legend = FALSE) +
    geom_point(size = 3, show.legend = FALSE) + 
    ggtitle("Win Percentage vs Payroll") + 
    ylab("Average Win Percentage") + 
    xlab("Average Payroll [millions]") + 
    theme_gray() + 
    theme(
        legend.position = "top",
        plot.title = element_text(size = 25, hjust = 0.5, face="bold"),
        axis.title = element_text(size=20),
        axis.title.x = element_text(margin = margin(t = 10)),
        axis.title.y = element_text(margin = margin(r = 15))
    )
filename = "fig6.png"
ggsave(filename, p, width=15, height = 8)
print_plot(filename)

<center style="font-size: 17px"><b>Figure 6</b>: Scatter plot of average `pct_win` vs average `payroll`  from <i>Table 2c</i> divided by 4 quadrants based on median (labeled with top 3 most efficient teams)</center>

##### In what sense do these three teams appear efficient in the previous plot?

Reinforcing the observation earlier, based on the **Figure 6**, the top three most efficient teams—*Miami Marlins*, *Oakland Athletics*, and *Tampa Bay Rays*—are the teams that spent less on `payroll` and achieve more winning output relative to the spending. These are the teams that are raw for the taking in terms of investment for they aren't spending as much to their players but still manage to maximize their capabilities. It is best to think of this as maximizing, and even optimizing players to the point that their output becomes relatively more than their market value (may it be because of coaching or culture)—thus efficiency.